In [10]:
import os 
os.chdir("../..")
from MSML.src.math_spec_mapping import (load_from_json, create_action_chains_graph, write_out_boundary_actions,
                      write_out_policies, write_out_mechanisms, load_svg_graphviz, write_basic_report_full,
                      write_action_chain_reports, write_spec_tree, create_parameter_impact_table,
                     write_entity_reports, write_wiring_report, write_entity_markdown_report,
                     write_all_markdown_reports)
os.chdir("Quai-Macro-Model/Spec")

from IPython.display import display, display_html
from copy import deepcopy
from src import math_spec_json
import numpy as np

ms = load_from_json(deepcopy(math_spec_json))

state_base = {'Qi Supply': None,
 'Quai Supply': None,
 'Block Number': None,
 'Block Difficulty': 6000,
 "Number of Regions":2,
"Zones per Region":2,
 'Current Hash Difference': None,
 'Historical Converted Qi': None,
 'Historical Converted Quai': None,
 'Historical Mined Ratio': None,
 'Historical Qi Hash': None,
 'Historical Quai Hash': None,
 'K Qi': None,
 'K Quai': None,
 'Quai Price': None,
 'Qi Price': 1,
 'Simulation History Log': None,
 'Dummy': None}
params_base = {'Minimum Quai Conversion Amount': None,
 'Minimum Qi Conversion Amount': None,
 'Asset Return Parameterization': None,
 'PID Parameterization': None,
 'Initial Block Difficulty': None,
 "FP Controller Update Policy": 'Linear Controller Policy',
 "Block Difficulty Multiples": {"Prime": 1,
                                         "Region": .5,
                                         "Zone": .25},
'Target Mining Time': 2}
msi = ms.build_implementation(params_base)

Placeholder has no control action options
Controller Update Control Action has no control action options
Conversions Boundary Action has no boundary action options
Price Movements Boundary Action has no boundary action options
No python implementation for Block Reward Policy / Block Reward Policy V1. To fix this, go to Implementations/Python/Policies and add Block Reward Policy V1
No python implementation for Mining Payment Policy / Deterministic Mining Payment Policy. To fix this, go to Implementations/Python/Policies and add Deterministic Mining Payment Policy
No python implementation for Conversions Policy / Block Reward Ratio Conversion Policy. To fix this, go to Implementations/Python/Policies and add Block Reward Ratio Conversion Policy
Price Movements Policy has no policy options
No python implementation for Controller Update Policy / Linear Controller Policy. To fix this, go to Implementations/Python/Policies and add Linear Controller Policy
No python implementation for Increme

# Hashpower Comparisons

The current dummy model multiplies the Qi price by 1000. We display a low, medium and high scenario below to illustrate when hashpower is not enough, just right and too high and how difficulty adjusts as well as what the spaces look like.

The target mining time is set to 2 seconds.

## Low Hashpower

In [13]:
np.random.seed(0)
state_i = deepcopy(state_base)
state_i["Qi Price"] = 5
state, params = msi.prepare_state_and_params(state_i, params_base, state_preperation_functions=[], parameter_preperation_functions=[])
spaces = msi.boundary_actions['Mine Block Boundary Action'](state, params, [])
print(spaces)
print()
spaces = msi.policies["Mining Policy"](state, params, spaces)
print(spaces)
print()

[{'Aggregate Hashpower': 5000, 'Blocks to Mine': [{'Difficulty': 1638, 'Block Type': 'Zone'}, {'Difficulty': 1530, 'Block Type': 'Zone'}, {'Difficulty': 3150, 'Block Type': 'Region'}, {'Difficulty': 1678, 'Block Type': 'Zone'}, {'Difficulty': 1647, 'Block Type': 'Zone'}, {'Difficulty': 2857, 'Block Type': 'Region'}, {'Difficulty': 6292, 'Block Type': 'Prime'}], 'Block Hash Cumulative Sum': array([ 1638,  3168,  6318,  7996,  9643, 12500, 18792])}]

[[{'Difficulty': 1638, 'Block Type': 'Zone'}, {'Difficulty': 1530, 'Block Type': 'Zone'}, {'Difficulty': 3150, 'Block Type': 'Region'}, {'Difficulty': 1678, 'Block Type': 'Zone'}, {'Difficulty': 1647, 'Block Type': 'Zone'}], [{'Difficulty': 2714, 'Block Type': 'Region'}, {'Difficulty': 5977, 'Block Type': 'Prime'}]] [2, 1.7382] 5700.0
None



## Medium Hashpower

In [14]:
np.random.seed(0)
state_i = deepcopy(state_base)
state_i["Qi Price"] = 10
state, params = msi.prepare_state_and_params(state_i, params_base, state_preperation_functions=[], parameter_preperation_functions=[])
spaces = msi.boundary_actions['Mine Block Boundary Action'](state, params, [])
print(spaces)
print()
spaces = msi.policies["Mining Policy"](state, params, spaces)
print(spaces)
print()

[{'Aggregate Hashpower': 10000, 'Blocks to Mine': [{'Difficulty': 1638, 'Block Type': 'Zone'}, {'Difficulty': 1530, 'Block Type': 'Zone'}, {'Difficulty': 3150, 'Block Type': 'Region'}, {'Difficulty': 1678, 'Block Type': 'Zone'}, {'Difficulty': 1647, 'Block Type': 'Zone'}, {'Difficulty': 2857, 'Block Type': 'Region'}, {'Difficulty': 6292, 'Block Type': 'Prime'}], 'Block Hash Cumulative Sum': array([ 1638,  3168,  6318,  7996,  9643, 12500, 18792])}]

[[{'Difficulty': 1638, 'Block Type': 'Zone'}, {'Difficulty': 1530, 'Block Type': 'Zone'}, {'Difficulty': 3150, 'Block Type': 'Region'}, {'Difficulty': 1678, 'Block Type': 'Zone'}, {'Difficulty': 1647, 'Block Type': 'Zone'}, {'Difficulty': 2857, 'Block Type': 'Region'}, {'Difficulty': 6292, 'Block Type': 'Prime'}]] [1.8792] 6000
None



## High Hashpower

In [15]:
np.random.seed(0)
state_i = deepcopy(state_base)
state_i["Qi Price"] = 100
state, params = msi.prepare_state_and_params(state_i, params_base, state_preperation_functions=[], parameter_preperation_functions=[])
spaces = msi.boundary_actions['Mine Block Boundary Action'](state, params, [])
print(spaces)
print()
spaces = msi.policies["Mining Policy"](state, params, spaces)
print(spaces)
print()

[{'Aggregate Hashpower': 100000, 'Blocks to Mine': [{'Difficulty': 1638, 'Block Type': 'Zone'}, {'Difficulty': 1530, 'Block Type': 'Zone'}, {'Difficulty': 3150, 'Block Type': 'Region'}, {'Difficulty': 1678, 'Block Type': 'Zone'}, {'Difficulty': 1647, 'Block Type': 'Zone'}, {'Difficulty': 2857, 'Block Type': 'Region'}, {'Difficulty': 6292, 'Block Type': 'Prime'}], 'Block Hash Cumulative Sum': array([ 1638,  3168,  6318,  7996,  9643, 12500, 18792])}]

[[{'Difficulty': 1638, 'Block Type': 'Zone'}, {'Difficulty': 1530, 'Block Type': 'Zone'}, {'Difficulty': 3150, 'Block Type': 'Region'}, {'Difficulty': 1678, 'Block Type': 'Zone'}, {'Difficulty': 1647, 'Block Type': 'Zone'}, {'Difficulty': 2857, 'Block Type': 'Region'}, {'Difficulty': 6292, 'Block Type': 'Prime'}]] [0.18792] 6300.0
None

